# Dask: Parallel and distributed Numpy, Pandas, and scikit-learn

By now, you should be very familiar with Numpy and Pandas, and at least a bit familiar with scikit-learn.  These are all great libraries.  But what if they could be faster?

Enter Dask.  Dask is a very cool library that essentially re-implement most of Numpy, Pandas, scikit-learn, and a few other libraries, but does so in a way that's natively parallelizeable, and can be easily distributed over a whole cluster of computers if needed.  It's not quite a drop-in replacement, but it's not too far from one.

Dask is mostly designed for working with _very_ large datasets.  As in, too large to fit in RAM.  If your dataset fits very nicely into RAM, you probably won't see much benefit to using Dask over Pandas (in fact, the Dask documentation recommends against this).  If you're not doing large amounts of highly parallelizeable computations on your data, you probably also won't see a huge benefit.  But, if you have a truly huge amount of data to process, Dask can absolutely save the day.

In my experience: if you're sticking with educational data and Learning Analytics, you will never _need_ a tool like Dask.  But it can still be useful to know about, because maybe there will come a day when you need to process a few hundred gigabytes of CSV files!

Install Dask with:

```bash
conda install dask
```

# Dask Quickstart

Code-wise, Dask is very simple if you already know Numpy and Pandas and scikit-learn.  You use a slightly different import (e.g.: `import dask.dataframe as ddf` rather than `import pandas as pd`), do your manipulations on the `DataFrame`/`array`/etc like you would normally, then finish it off with a call to the `.compute()` method (e.g.: `dask.dataframe.compute()`). 

There are two big ideas Dask uses to parallelize its work.  The first is a _compute graph,_ and the second is _chunking._

Chunking is the easier of these to explain.  When you use Dask's implementation of `DataFrame`s, it actually creates multiple Pandas `DataFrames` that each contain separate chunks of the data.  It run whatever calculations you request on these chunks, in paralle, then combines them.  Same thing for Numpy `array`s and other data structures.

The compute graph is a bit more complex, but it's still pretty straightforward.  When you ask Dask to do something to any of its data structures, it doesn't immediately do it.  E.g.: if you ask it to calculate the maximum value of some column, it will hold off on actually calculating that.  It will build up a list of all the things you're asking it to calculate.  The final step in your process is always to call `my_data.compute()`, which tells Dask to start executing all those steps.

The reason Dask holds off on actually running any code is that this lets it figure out more efficient ways to go about calculating things.  If it knows all the steps you need to take, it can often find more efficient approaches, or just be smarter about how it splits up the data across different parallel chunks.  E.g.: if you tell it to calculate the maximum of some column in a `DataFrame`, but you never use that value for anything later, Dask might realize it doesn't need to calculate that, and skip that step entirely.  This is the same thing that a lot of compiled programming languages do: they read your entire program before they start executing any code, because they might be able to find faster ways to get to the exact same answer.  (Python, being interpreted, does not do this; it executes code in a "dumb" way, one line at a time.  This has many benefits, but speed is not one of them).

Let's do some data manipulation using Dask, and compare it to Pandas.  Let's do some language-based analysis this time.  We'll use the public [Google Ngram data](https://storage.googleapis.com/books/ngrams/books/datasetsv3.html), which provide counts of how often different words, pairs of words, triplets of words, etc. appear across the Google Book corpus.  Let's focus on just one question: _what words most often follow the phrase "intend to" in a sentence?_

A few notes about the Google Ngram data.  First, the files are _very_ large (often hundreds of megabytes per file, compressed; uncompressed, it's gigabytes per file).  Second, the files are a bit hard to navigate if you're looking for specific words, due to how they're sorted (alphabetically, but case-sensitive; so "INTEND" sorts before "Intent" sorts before "intend").  We're going to grab only the files that contain "intend" in all lowercase.

# `dask.dataframe`: Parallel/distributed Pandas `DataFrame`s

In [1]:
# Download the data if it doesn't already exist on your machine.
# NOTE: this file is LARGE.  You'll need a bit over 2gb of free space.
import gzip
import os
import requests

if not os.path.isfile("dask_demo_data.csv"):
    # data = requests.get("https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2019_msa_sec.csv")
    data = requests.get("https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2019_msa_sec_fac.csv")
    print(len(data.content))
    with open("dask_demo_data.csv", "wb") as OUT:
        OUT.write(data.content)

In [2]:
%%time

import pandas as pd

pandas_df = pd.read_csv(
    "dask_demo_data.csv",
    usecols=["year", "msa", "sector", "firms"],
    dtype={"year": int, "msa": "category", "sector": "category"},
)
pandas_df["firms"] = pandas_df["firms"].replace({"(D)": 0, "(X)": 0})
pandas_df["firms"] = pandas_df["firms"].astype(int)
res = pandas_df.groupby(["year", "msa", "sector"])["firms"].mean() 
print(res)

year  msa    sector
1978  10100  11         0.0
             21         0.0
             22         0.8
             23        26.4
             31-33      6.6
                       ... 
2019  49820  61         0.0
             62         2.6
             71         0.0
             72         3.0
             81         0.8
Name: firms, Length: 731766, dtype: float64
CPU times: total: 4.42 s
Wall time: 4.48 s


In [3]:
%%time

import dask.dataframe as ddf

dask_df = ddf.read_csv(
    "dask_demo_data.csv",
    usecols=["year", "msa", "sector", "firms"],
    dtype={"year": int, "msa": "category", "sector": "category"},
)
dask_df["firms"] = dask_df["firms"].replace({"(D)": 0, "(X)": 0})
dask_df["firms"] = dask_df["firms"].astype(int)
res = dask_df.groupby(["year", "msa", "sector"])["firms"].mean()
# This shows no results because nothing has been computed yet.
print(res)

# call .compute() to actually run the computation 
print(res.compute())

Dask Series Structure:
npartitions=1
    float64
        ...
Name: firms, dtype: float64
Dask Name: truediv, 57 tasks
year  msa    sector
1978  10100  11         0.0
             21         0.0
             22         0.8
             23        26.4
             31-33      6.6
                       ... 
2019  49820  61         0.0
             62         2.6
             71         0.0
             72         3.0
             81         0.8
Name: firms, Length: 731766, dtype: float64
CPU times: total: 10.5 s
Wall time: 5.17 s


In this case, the Dask version ran slower than the pure Pandas version.  This is expected, even for a pretty large dataset like this.  However, if we had to run a _lot_ of parallelizeable operations over our data, Dask would probably pull ahead.

I've used Dask on occasion to process datasets measuring in the hundreds of gigabytes, because Pandas and other tools just simply would not work at that scale.  So Dask is very good to know about, but you generally don't need to reach for it unless you don't have enough RAM to do what you want to.

# `dask.array`: Numpy `array`s

Unlike `dask.dataframe`, I see much more consistent speedups when using `dask.array`--the re-implementation of Numpy's `array`s.  Most of the work I do involves lots of elementwise operations over Numpy arrays, which is easily parallelizeable, but your mileage may vary.  Like with `dask.dataframe`, `dask.array` is basically a drop-in replacement for most of Numpy.  Note, however, that certain installations of Numpy--like those from conda--might already be using highly parallelized implementations of some algorithms behind the scenes.  So you might, again, see Dask run a lot slower for the same kinds of operations.

In [4]:
%%time
import numpy as np
arr = np.random.random(size=(100_000, 1000))
arr = np.log1p(arr)
maxval = arr.max()
print(maxval)

0.6931471771616475
CPU times: total: 1.7 s
Wall time: 1.7 s


In [5]:
%%time
import dask.array as da
arr = da.random.random(size=(100_000, 10_000))
arr = da.log1p(arr)
res = arr.max()
print(res.compute())

0.6931471794982909
CPU times: total: 39.4 s
Wall time: 3.87 s


Note that in the above examples, the dot product step should really be run on a GPU if one is available and maximum performance is needed.

# dask.dask_ml: scikit-learn

The scikit-learn re-implementation in `dask.dask_ml` actually adds quite a lot of new stuff.  A lot of this is in the hyperparameter tuning tools.  Dask adds some tools that are much "smarter" than scikit-learn's `RandomizedSearchCV()` and `GridSearchCV()` objects.  E.g., Dask has algorithms that will partially fit a bunch of model candidates, and quickly discard ones that show no immediate improvement.  A lot of Dask's parameter tuning tools are also designed to _avoid re-doing work._  Consider, e.g., the following scikit-learn pipeline:

```python
model = Pipeline([
    ("feature selection", SelectKBest(k=1000)),
    ("dimensionality reduction", TSNE()),
    ("scaler", StandardScaler()),
    ("classifier", LogisticRegression()),
])

gs = GridSearchCV(
    estimator=model,
    param_grid={
        "dimensionality reduction": [TSNE(), PCA(), None],
        "classifier_c": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]
    }
)
gs.fit(x, y)
```

The `GridSearchCV()` object will re-use the exact same cross-validation folds for every single combination of parameters.  It's also going to completely re-fit the `Pipeline()` from scratch, every single time.  This means the SelectKBest() and TSNE() are going to get re-fit to the exact same data a _lot,_ and that could be a slow process.

Dask ML will avoid doing this.  It will do a lot of work to save the SelectKBest(), TSNE(), etc. steps after fitting, and to only re-fit when there's brand new data.  If it sees the same data come through--e.g., the same cross-validation fold--it will re-use the version of the step already fitted to that data.

Dask ML also re-implements some of scikit-learn's models, and adds support for a few extra ones like XGBoost.

Dask ML's differences from scikit-learn are pretty much just in the support for distributed/cluster computing (which I can't demonstrate, since I don't have access to any clusters), and in a few specific new models they add.  The code looks almost indistinguishable from scikit-learn otherwise, so you'll have to go experiment with it yourself a bit.